https://github.com/Hukex/Model-Blurfaces Repositorio donde se ejecutan este notebook con todos los datos y modelos generados

In [2]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
model = YOLO("model.pt") #https://huggingface.co/arnabdhar/YOLOv8-Face-Detection

image_path = "foto.png"
im2 = cv2.imread(image_path)

results = model.predict(source=im2, save=False, save_txt=False)

img_all_blurred = cv2.medianBlur(im2, 25)

mask_img = np.zeros(im2.shape, dtype='uint8')
for box in results[0].boxes:
    x_min, y_min, x_max, y_max = map(int, box.xyxy.tolist()[0])
    w, h = x_max - x_min, y_max - y_min
    ellipse_center = ((x_min + x_max) // 2, (y_min + y_max) // 2)
    axes = (w // 2, h // 2)
    cv2.ellipse(mask_img, ellipse_center, axes, 0, 0, 360, (255, 255, 255), -1)

im2 = np.where(mask_img > 0, img_all_blurred, im2)

base_name, ext = os.path.splitext(image_path)
output_path = f"{base_name}_blurred{ext}"

cv2.imwrite(output_path, im2)

In [3]:
class Specificity(tf.keras.metrics.Metric):
    def __init__(self, name='specificity', **kwargs):
        super(Specificity, self).__init__(name=name, **kwargs)
        self.true_negatives = self.add_weight(name='tn', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.bool)
        y_pred = tf.cast(y_pred > 0.5, tf.bool)
        self.true_negatives.assign_add(tf.reduce_sum(
            tf.cast(tf.logical_not(y_true) & tf.logical_not(y_pred), tf.float32)))
        self.false_positives.assign_add(tf.reduce_sum(
            tf.cast(tf.logical_not(y_true) & y_pred, tf.float32)))

    def result(self):
        specificity = self.true_negatives / \
            (self.true_negatives + self.false_positives)
        return specificity

    def reset_state(self):
        self.true_negatives.assign(0)
        self.false_positives.assign(0)

# Definir la métrica F1-Score


class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [16]:
yolo_model = YOLO("model.pt")
print("Modelo YOLO cargado correctamente.")

classification_model = tf.keras.models.load_model("modelo_ajustado_1520.h5", custom_objects={
    'Specificity': Specificity,
    'F1Score': F1Score
})
print("Modelo de clasificación cargado correctamente.")

img_height = 200
img_width = 200

def preprocess_image(img, save_path=None):
    img_resized = cv2.resize(img, (img_width, img_height))
    img_normalized = img_resized.astype('float32') / 255.0
    img_expanded = np.expand_dims(img_normalized, axis=0)
    
    if save_path:
        cv2.imwrite(save_path, (img_normalized * 255).astype('uint8'))
    
    return img_expanded

image_path = "foto2.png"
im2 = cv2.imread(image_path)
if im2 is not None:
    print("Imagen cargada correctamente.")
else:
    print("Error al cargar la imagen.")

base_name, ext = os.path.splitext(image_path)
preprocessed_dir = f"{base_name}_preprocessed"
os.makedirs(preprocessed_dir, exist_ok=True)

results = yolo_model.predict(source=im2, save=False, save_txt=False)
print(f"Detecciones YOLO: {len(results[0].boxes)} caras detectadas.")

img_all_blurred = cv2.medianBlur(im2, 25)

# Crear una máscara para las áreas que necesitan desenfoque
mask_img = np.zeros(im2.shape, dtype='uint8')

for i, box in enumerate(results[0].boxes):
    x_min, y_min, x_max, y_max = map(int, box.xyxy.tolist()[0])
    face_img = im2[y_min:y_max, x_min:x_max]
    
    preprocessed_face_path = os.path.join(preprocessed_dir, f"preprocessed_face_{i}.png")
    preprocessed_face = preprocess_image(face_img, save_path=preprocessed_face_path)
    print(f"Preprocessed face {i} shape: {preprocessed_face.shape}")
    
    prediction = classification_model.predict(preprocessed_face)
    is_minor = prediction[0][0] > 0.5 
    
    print(f"Predicción: {prediction[0][0]}, Es menor: {is_minor}")
    
    if is_minor:
        w, h = x_max - x_min, y_max - y_min
        ellipse_center = ((x_min + x_max) // 2, (y_min + y_max) // 2)
        axes = (w // 2, h // 2)
        cv2.ellipse(mask_img, ellipse_center, axes, 0, 0, 360, (255, 255, 255), -1)

im2 = np.where(mask_img > 0, img_all_blurred, im2)

output_path = f"{base_name}_blurred{ext}"

cv2.imwrite(output_path, im2)
print(f"Imagen guardada en {output_path}")

Modelo YOLO cargado correctamente.
Modelo de clasificación cargado correctamente.
Imagen cargada correctamente.

0: 448x640 11 FACEs, 44.2ms
Speed: 4.0ms preprocess, 44.2ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Detecciones YOLO: 11 caras detectadas.
Preprocessed face 0 shape: (1, 200, 200, 3)
1/1 [==============================] - 1s 530ms/step
Predicción: 0.05824455991387367, Es menor: False
Preprocessed face 1 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 17ms/step
Predicción: 0.9972143769264221, Es menor: True
Preprocessed face 2 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 17ms/step
Predicción: 0.9630131125450134, Es menor: True
Preprocessed face 3 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 33ms/step
Predicción: 0.9993026256561279, Es menor: True
Preprocessed face 4 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 14ms/step
Predicción: 0.9570925235748291, Es menor: True
Prepr

In [15]:
# Cargar modelo YOLO
yolo_model = YOLO("model.pt")
print("Modelo YOLO cargado correctamente.")

# Lista de modelos de clasificación
model_names = [
    "model_lr_0.0001_batch_16_batchnorm_True_Transform_1",  # 0.2
    "10_20_ajustado",  # 0.4
    "15_20_ajustado",  # 0.4
    "20_20_ajustado"  # 0.4
]

model_thresholds = {
    "model_lr_0.0001_batch_16_batchnorm_True_Transform_1": 0.2,
    "10_20_ajustado": 0.4,
    "15_20_ajustado": 0.4,
    "20_20_ajustado": 0.4
}

# Configuración de la imagen
img_height = 200
img_width = 200

def preprocess_image(img, save_path=None):
    img_resized = cv2.resize(img, (img_width, img_height))
    img_normalized = img_resized.astype('float32') / 255.0
    img_expanded = np.expand_dims(img_normalized, axis=0)

    if save_path:
        cv2.imwrite(save_path, (img_normalized * 255).astype('uint8'))

    return img_expanded

# Procesar múltiples imágenes
image_names = ['foto.png'] + [f"foto{i}.png" for i in range(2, 9)]  # Lista de nombres de imágenes desde foto.png hasta foto8.png
for image_path in image_names:
    im2 = cv2.imread(image_path)
    if im2 is not None:
        print(f"Imagen {image_path} cargada correctamente.")
    else:
        print(f"Error al cargar la imagen {image_path}.")
        continue

    base_name, ext = os.path.splitext(image_path)
    preprocessed_dir = f"{base_name}_preprocessed"
    output_dir = f"{base_name}_outputs"
    os.makedirs(preprocessed_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    results = yolo_model.predict(source=im2, save=False, save_txt=False)
    print(f"Detecciones YOLO: {len(results[0].boxes)} caras detectadas.")

    # Iterar sobre todos los modelos y realizar predicciones
    for model_name in model_names:
        model_path = f"{model_name}.h5"
        threshold = model_thresholds[model_name]

        if os.path.exists(model_path):
            print(f"Cargando {model_path}...")
            classification_model = tf.keras.models.load_model(model_path, custom_objects={
                'Specificity': Specificity,
                'F1Score': F1Score
            })
            print(f"Modelo {model_name} cargado correctamente.")

            img_all_blurred = cv2.medianBlur(im2, 25)
            mask_img = np.zeros(im2.shape, dtype='uint8')

            for i, box in enumerate(results[0].boxes):
                x_min, y_min, x_max, y_max = map(int, box.xyxy.tolist()[0])
                face_img = im2[y_min:y_max, x_min:x_max]

                preprocessed_face_path = os.path.join(
                    preprocessed_dir, f"preprocessed_face_{i}.png")
                preprocessed_face = preprocess_image(
                    face_img, save_path=preprocessed_face_path)
                print(f"Preprocessed face {i} shape: {preprocessed_face.shape}")

                prediction = classification_model.predict(preprocessed_face)
                is_minor = prediction[0][0] > threshold

                print(f"Predicción: {prediction[0][0]}, Es menor: {is_minor}")

                if is_minor:
                    w, h = x_max - x_min, y_max - y_min
                    ellipse_center = ((x_min + x_max) // 2, (y_min + y_max) // 2)
                    axes = (w // 2, h // 2)
                    cv2.ellipse(mask_img, ellipse_center, axes,
                                0, 0, 360, (255, 255, 255), -1)

            final_image = np.where(mask_img > 0, img_all_blurred, im2)
            output_path = os.path.join(
                output_dir, f"{base_name}_{model_name}_blurred{ext}")

            cv2.imwrite(output_path, final_image)
            print(f"Imagen guardada en {output_path}")

Modelo YOLO cargado correctamente.
Imagen foto.png cargada correctamente.

0: 384x640 5 FACEs, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Detecciones YOLO: 5 caras detectadas.
Cargando model_lr_0.0001_batch_16_batchnorm_True_Transform_1.h5...
Modelo model_lr_0.0001_batch_16_batchnorm_True_Transform_1 cargado correctamente.
Preprocessed face 0 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 136ms/step
Predicción: 0.7317445278167725, Es menor: True
Preprocessed face 1 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 16ms/step
Predicción: 0.5827840566635132, Es menor: True
Preprocessed face 2 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 16ms/step
Predicción: 0.12390947341918945, Es menor: False
Preprocessed face 3 shape: (1, 200, 200, 3)
1/1 [==============================] - 0s 11ms/step
Predicción: 0.24294811487197876, Es menor: True
Preprocessed face 4 shape: (1, 200, 200,